<h1>Multiple Linear Regression</h1>


Import the libraries and set the random seed.


In [4]:
# Import the libraries and set the random seed
import torch
from torch import nn
torch.manual_seed(1)

<h2 id="Prediction">Prediction</h2>


Set weight and bias.


In [5]:
# Set the weight and bias

w = torch.tensor([[2.0], [3.0]], requires_grad=True)
b = torch.tensor([[1.0]], requires_grad=True)

Define the parameters. <code>torch.mm</code> uses matrix multiplication instead of scaler multiplication.


In [6]:
# Define Prediction Function

def forward(x):
    yhat = torch.mm(x, w) + b
    return yhat

The function <code>forward</code> implements the following equation:


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter2/2.6.1_matrix_eq.png" width="600" alt="Matrix Linear Regression">


If we input a <i>1x2</i> tensor, because we have a <i>2x1</i> tensor as <code>w</code>, we will get a <i>1x1</i> tensor: 


In [7]:
# Calculate yhat

x = torch.tensor([[1.0, 2.0]])
yhat = forward(x)
print("The result: ", yhat)

The result:  tensor([[9.]], grad_fn=<AddBackward0>)


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter2/2.6.1example.png" width="300" alt="Linear Regression Matrix Sample One">


# Each row of the following tensor represents a sample:


In [8]:
# Sample tensor X

X = torch.tensor([[1.0, 1.0], [1.0, 2.0], [1.0, 3.0]])

In [9]:
# Make the prediction of X 

yhat = forward(X)
print("The result: ", yhat)

The result:  tensor([[ 6.],
        [ 9.],
        [12.]], grad_fn=<AddBackward0>)


<h2 id="Linear">Class Linear</h2>


We can use the linear class to make a prediction. You'll also use the linear class to build more complex models.


Let us create a model.


In [10]:
# Make a linear regression model using build-in function

model = nn.Linear(2, 1)

Make a prediction with the first sample:


In [11]:
# Make a prediction of x

yhat = model(x)
print("The result: ", yhat)

The result:  tensor([[-0.3969]], grad_fn=<AddmmBackward0>)


In [12]:
model.state_dict()

OrderedDict([('weight', tensor([[ 0.3643, -0.3121]])),
             ('bias', tensor([-0.1371]))])

In [13]:
# Make a prediction of X

yhat = model(X)
print("The result: ", yhat)

The result:  tensor([[-0.0848],
        [-0.3969],
        [-0.7090]], grad_fn=<AddmmBackward0>)


The function performs matrix multiplication as shown in this image:


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter2/2.6.1multi_sample_example.png" width="600" alt="Linear Regression Matrix Sample One">


<!--Empty Space for separating topics-->


<h2 id="Cust">Build Custom Modules </h2>


Now, you'll build a custom module. You can make more complex models by using this method later. 


In [14]:
#Creating custom linear_regression class

class linear_regression(nn.Module):
    
    #Constructor
    def __init__(self, input_size, output_size):
        super(linear_regression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        
    #Prediction function  
    def forward(self, x):
        yhat = self.linear(x)
        return yhat

Build a linear regression object. The input feature size is two. 


In [15]:
model = linear_regression(2, 1)

This will input the following equation:


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter2/2.6.1_matrix_eq.png" width="600" alt="Matrix Linear Regression">


You can see the randomly initialized parameters by using the <code>parameters()</code> method:


In [16]:
# Print model parameters

print("The parameters: ", list(model.parameters()))

The parameters:  [Parameter containing:
tensor([[ 0.3319, -0.6657]], requires_grad=True), Parameter containing:
tensor([0.4241], requires_grad=True)]


You can also see the parameters by using the <code>state_dict()</code> method:


In [17]:
# Print model parameters

print("The parameters: ", model.state_dict())

The parameters:  OrderedDict([('linear.weight', tensor([[ 0.3319, -0.6657]])), ('linear.bias', tensor([0.4241]))])


Now we input a 1x2 tensor, and we will get a 1x1 tensor.


In [18]:
# Make a prediction of x

yhat = model(x)
print("The result: ", yhat)

The result:  tensor([[-0.5754]], grad_fn=<AddmmBackward0>)


The shape of the output is shown in the following image: 


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter2/2.6.1_matrix_eq.png" width="600" alt="Matrix Linear Regression">


Make a prediction for multiple samples:


In [19]:
# Make a prediction of X

yhat = model(X)
print("The result: ", yhat)

The result:  tensor([[ 0.0903],
        [-0.5754],
        [-1.2411]], grad_fn=<AddmmBackward0>)


The shape is shown in the following image: 


<img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0110EN/notebook_images%20/chapter2/2.6.1Multi_sample.png" width="600" alt="Multiple Samples Linear Regression">
